In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier, StackingClassifier
import optuna
import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading and Initial Inspection
# Training data
train_categorical = pd.read_excel("widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_quantitative = pd.read_excel("widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_connectome = pd.read_csv("widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_solutions = pd.read_excel("widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# Test data
test_categorical = pd.read_excel("widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_quantitative = pd.read_excel("widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
test_connectome = pd.read_csv("widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")

# 2. Data Preprocessing
# Check for matching participant IDs
def check_and_merge_data(cat_df, quant_df, conn_df, solutions_df=None):
    # Ensure participant_id is the index
    cat_df = cat_df.set_index('participant_id') if 'participant_id' in cat_df.columns else cat_df
    quant_df = quant_df.set_index('participant_id') if 'participant_id' in quant_df.columns else quant_df
    conn_df = conn_df.set_index('participant_id') if 'participant_id' in conn_df.columns else conn_df
    
    # Find common participant IDs
    common_ids = set(cat_df.index).intersection(set(quant_df.index)).intersection(set(conn_df.index))
    
    if solutions_df is not None:
        solutions_df = solutions_df.set_index('participant_id') if 'participant_id' in solutions_df.columns else solutions_df
        common_ids = common_ids.intersection(set(solutions_df.index))
        
    print(f"Found {len(common_ids)} common participant IDs")
    
    # Filter datasets to common IDs
    cat_df = cat_df.loc[common_ids]
    quant_df = quant_df.loc[common_ids]
    conn_df = conn_df.loc[common_ids]
    
    # Merge all data
    merged_df = pd.concat([cat_df, quant_df, conn_df], axis=1)
    
    if solutions_df is not None:
        solutions_df = solutions_df.loc[common_ids]
        return merged_df, solutions_df
    else:
        return merged_df

# Merge training data
train_data, train_targets = check_and_merge_data(
    train_categorical, train_quantitative, train_connectome, train_solutions
)

# Merge test data
test_data = check_and_merge_data(
    test_categorical, test_quantitative, test_connectome
)

# Extract target variables
y_adhd = train_targets['ADHD_Outcome']
y_sex = train_targets['Sex_F']

# 3. Feature Engineering

# Create combined stratification column
stratify_col = y_adhd.astype(str) + '_' + y_sex.astype(str)

# Function to extract features from connectome data
def create_connectome_features(connectome_df):
    # Identify connectivity columns
    conn_cols = [col for col in connectome_df.columns if col.startswith('roi')]
    
    # Basic statistics
    conn_features = pd.DataFrame(index=connectome_df.index)
    conn_features['mean_connectivity'] = connectome_df[conn_cols].mean(axis=1)
    conn_features['std_connectivity'] = connectome_df[conn_cols].std(axis=1)
    conn_features['max_connectivity'] = connectome_df[conn_cols].max(axis=1)
    conn_features['min_connectivity'] = connectome_df[conn_cols].min(axis=1)
    
    # Extract regions from column names and aggregate by region
    regions = set()
    for col in conn_cols:
        parts = col.split('_')
        if len(parts) >= 3:
            regions.add(parts[1])
            regions.add(parts[2])
    
    # Create region-specific features
    for region in list(regions)[:30]:  # Limit to prevent feature explosion
        region_cols = [col for col in conn_cols if f'_{region}_' in col or f'_{region}' == col[-len(region)-1:]]
        if region_cols:
            conn_features[f'region_{region}_mean'] = connectome_df[region_cols].mean(axis=1)
            conn_features[f'region_{region}_std'] = connectome_df[region_cols].std(axis=1)
    
    # Create graph metrics
    # Calculate degree (number of strong connections)
    threshold = 0.5  # Arbitrary threshold for demonstration
    strong_connections = (connectome_df[conn_cols] > threshold).sum(axis=1)
    conn_features['degree'] = strong_connections
    
    return conn_features

# Apply feature engineering
train_conn_features = create_connectome_features(train_data)
test_conn_features = create_connectome_features(test_data)

# Split categorical and numerical features
cat_features = train_data.select_dtypes(include=['object', 'category']).columns.tolist()
num_features = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove any target columns that might be in the features
num_features = [col for col in num_features if col not in ['ADHD_Outcome', 'Sex_F']]

# Combine engineered features with original data
train_data = pd.concat([train_data, train_conn_features], axis=1)
test_data = pd.concat([test_data, test_conn_features], axis=1)

# 4. Create preprocessing pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ],
    remainder='drop'  # Drop columns not specified
)

# 5. Model Development and Hyperparameter Tuning
# Split data for validation
X_train, X_val, y_adhd_train, y_adhd_val, y_sex_train, y_sex_val = train_test_split(
    train_data, y_adhd, y_sex, 
    test_size=0.2, 
    random_state=42,
    stratify=stratify_col
)

# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(test_data)

# Apply SMOTE separately for sex and ADHD prediction
smote_sex = SMOTE(random_state=42)
X_train_sex_resampled, y_sex_train_resampled = smote_sex.fit_resample(X_train_processed, y_sex_train)

smote_adhd = SMOTE(random_state=42)
X_train_adhd_resampled, y_adhd_train_resampled = smote_adhd.fit_resample(X_train_processed, y_adhd_train)

# Define optimization function for sex model
def optimize_sex_model(trial):
    params = {
        'objective': 'binary',
        'metric': 'f1',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 5.0)
    }
    
    # Create model
    model = lgb.LGBMClassifier(**params, random_state=42)
    
    # Train model
    model.fit(X_train_sex_resampled, y_sex_train_resampled)
    
    # Make predictions
    preds = model.predict(X_val_processed)
    f1 = f1_score(y_sex_val, preds)
    
    return f1

# Define optimization function for ADHD model
def optimize_adhd_model(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0)
    }
    
    # Create model
    model = xgb.XGBClassifier(**params, random_state=42)
    
    # Train model
    model.fit(X_train_adhd_resampled, y_adhd_train_resampled)
    
    # Make predictions
    preds = model.predict(X_val_processed)
    f1 = f1_score(y_adhd_val, preds)
    
    return f1

# Run hyperparameter optimization
sex_study = optuna.create_study(direction='maximize')
sex_study.optimize(optimize_sex_model, n_trials=20)

adhd_study = optuna.create_study(direction='maximize')
adhd_study.optimize(optimize_adhd_model, n_trials=20)

# Train final models with best parameters
best_sex_params = sex_study.best_params
best_adhd_params = adhd_study.best_params

print("Best Sex Model Parameters:")
print(best_sex_params)
print("\nBest ADHD Model Parameters:")
print(best_adhd_params)

# Train final sex model
sex_model = lgb.LGBMClassifier(**best_sex_params, random_state=42)
sex_model.fit(X_train_sex_resampled, y_sex_train_resampled)

# Train final ADHD model
adhd_model = xgb.XGBClassifier(**best_adhd_params, random_state=42)
adhd_model.fit(X_train_adhd_resampled, y_adhd_train_resampled)

# 6. Create specialized female ADHD model
female_indices = (y_sex_train == 1)
X_female_train = X_train_processed[female_indices]
y_female_adhd_train = y_adhd_train[female_indices]

# Apply SMOTE for balance
smote_female = SMOTE(random_state=42)
X_female_train_resampled, y_female_adhd_train_resampled = smote_female.fit_resample(X_female_train, y_female_adhd_train)

# Train specialized model
female_adhd_model = xgb.XGBClassifier(
    learning_rate=0.1,
    max_depth=6,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=2.0,
    random_state=42
)
female_adhd_model.fit(X_female_train_resampled, y_female_adhd_train_resampled)

# 7. Create stacked ensemble model
def create_ensemble():
    # First level models
    estimators = [
        ('xgb', xgb.XGBClassifier(random_state=42)),
        ('lgb', lgb.LGBMClassifier(random_state=42)),
        ('xgb_tuned', xgb.XGBClassifier(**best_adhd_params, random_state=42))
    ]
    
    # Final model
    final_estimator = xgb.XGBClassifier(learning_rate=0.1, random_state=42)
    
    # Create stacking ensemble
    stacked_model = StackingClassifier(
        estimators=estimators,
        final_estimator=final_estimator,
        cv=5,
        stack_method='predict_proba'
    )
    
    return stacked_model

# Train ensemble models
adhd_ensemble = create_ensemble()
adhd_ensemble.fit(X_train_adhd_resampled, y_adhd_train_resampled)

sex_ensemble = create_ensemble()
sex_ensemble.fit(X_train_sex_resampled, y_sex_train_resampled)

# 8. Prediction and threshold optimization
# Make probability predictions
sex_val_proba = sex_model.predict_proba(X_val_processed)[:, 1]
adhd_val_proba = adhd_model.predict_proba(X_val_processed)[:, 1]

# Also get predictions from ensembles
sex_ensemble_proba = sex_ensemble.predict_proba(X_val_processed)[:, 1]
adhd_ensemble_proba = adhd_ensemble.predict_proba(X_val_processed)[:, 1]

# For females, blend with specialized model predictions
sex_val_preds = (sex_val_proba >= 0.5).astype(int)
female_val_indices = np.where(sex_val_preds == 1)[0]

if len(female_val_indices) > 0:
    female_val_subset = X_val_processed[female_val_indices]
    female_adhd_proba = female_adhd_model.predict_proba(female_val_subset)[:, 1]
    
    # Blend predictions for females (70% specialized, 30% general)
    adhd_val_proba_blended = adhd_val_proba.copy()
    adhd_val_proba_blended[female_val_indices] = 0.3 * adhd_val_proba[female_val_indices] + 0.7 * female_adhd_proba

# Optimize thresholds
def find_optimal_threshold(y_true, y_proba):
    best_f1 = 0
    best_threshold = 0.5
    
    for threshold in np.arange(0.1, 0.9, 0.05):
        y_pred = (y_proba >= threshold).astype(int)
        current_f1 = f1_score(y_true, y_pred)
        
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_threshold = threshold
    
    return best_threshold, best_f1

# Find optimal thresholds
sex_threshold, sex_best_f1 = find_optimal_threshold(y_sex_val, sex_val_proba)
adhd_threshold, adhd_best_f1 = find_optimal_threshold(y_adhd_val, adhd_val_proba_blended)

print(f"Optimal Sex Threshold: {sex_threshold:.4f}, F1: {sex_best_f1:.4f}")
print(f"Optimal ADHD Threshold: {adhd_threshold:.4f}, F1: {adhd_best_f1:.4f}")

# Create final blended predictions
final_sex_val_proba = 0.6 * sex_val_proba + 0.4 * sex_ensemble_proba
final_adhd_val_proba = 0.5 * adhd_val_proba_blended + 0.5 * adhd_ensemble_proba

final_sex_val_preds = (final_sex_val_proba >= sex_threshold).astype(int)
final_adhd_val_preds = (final_adhd_val_proba >= adhd_threshold).astype(int)

# Evaluate final blended model
sex_final_f1 = f1_score(y_sex_val, final_sex_val_preds)
adhd_final_f1 = f1_score(y_adhd_val, final_adhd_val_preds)

print(f"Final Sex F1: {sex_final_f1:.4f}")
print(f"Final ADHD F1: {adhd_final_f1:.4f}")

# Calculate overall score (average of both F1 scores)
overall_f1 = (sex_final_f1 + adhd_final_f1) / 2
print(f"Overall F1 Score: {overall_f1:.4f}")

# 9. Apply to test set and generate submission
# Make predictions on test data
sex_test_proba = sex_model.predict_proba(X_test_processed)[:, 1]
adhd_test_proba = adhd_model.predict_proba(X_test_processed)[:, 1]

# Also get predictions from ensembles
sex_ensemble_test_proba = sex_ensemble.predict_proba(X_test_processed)[:, 1]
adhd_ensemble_test_proba = adhd_ensemble.predict_proba(X_test_processed)[:, 1]

# For females, blend with specialized model predictions
sex_test_preds = (sex_test_proba >= 0.5).astype(int)
female_test_indices = np.where(sex_test_preds == 1)[0]

if len(female_test_indices) > 0:
    female_test_subset = X_test_processed[female_test_indices]
    female_adhd_test_proba = female_adhd_model.predict_proba(female_test_subset)[:, 1]
    
    # Blend predictions for females (70% specialized, 30% general)
    adhd_test_proba_blended = adhd_test_proba.copy()
    adhd_test_proba_blended[female_test_indices] = 0.3 * adhd_test_proba[female_test_indices] + 0.7 * female_adhd_test_proba
else:
    adhd_test_proba_blended = adhd_test_proba

# Create final blended predictions
final_sex_test_proba = 0.6 * sex_test_proba + 0.4 * sex_ensemble_test_proba
final_adhd_test_proba = 0.5 * adhd_test_proba_blended + 0.5 * adhd_ensemble_test_proba

final_sex_test_preds = (final_sex_test_proba >= sex_threshold).astype(int)
final_adhd_test_preds = (final_adhd_test_proba >= adhd_threshold).astype(int)

# Create submission file
submission = pd.DataFrame({
    'participant_id': test_data.index,
    'ADHD_Outcome': final_adhd_test_preds,
    'Sex_F': final_sex_test_preds
})

submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")

# 10. Feature importance analysis
if hasattr(sex_model, 'feature_importances_'):
    # Get feature names after preprocessing
    feature_names = []
    for name, transformer, features in preprocessor.transformers_:
        if name == 'num':
            feature_names.extend(features)
        elif name == 'cat':
            ohe = transformer.named_steps['onehot']
            if hasattr(ohe, 'get_feature_names_out'):
                cat_features_transformed = ohe.get_feature_names_out(features)
                feature_names.extend(cat_features_transformed)
            else:
                # Fallback for older scikit-learn versions
                for cat in features:
                    feature_names.append(f'{cat}_encoded')
    
    # Plot sex model feature importance
    plt.figure(figsize=(15, 10))
    sex_importance = sex_model.feature_importances_
    if len(sex_importance) == len(feature_names):
        sorted_idx = np.argsort(sex_importance)[-20:]
        plt.barh(range(len(sorted_idx)), sex_importance[sorted_idx])
        plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
        plt.title('Sex Model: Top 20 Feature Importances')
        plt.tight_layout()
        plt.savefig('sex_feature_importance.png')
    
    # Plot ADHD model feature importance
    plt.figure(figsize=(15, 10))
    adhd_importance = adhd_model.feature_importances_
    if len(adhd_importance) == len(feature_names):
        sorted_idx = np.argsort(adhd_importance)[-20:]
        plt.barh(range(len(sorted_idx)), adhd_importance[sorted_idx])
        plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
        plt.title('ADHD Model: Top 20 Feature Importances')
        plt.tight_layout()
        plt.savefig('adhd_feature_importance.png') 